In [2]:
!git clone -b janina https://github.com/inttx/DLAM_SealedSurfaces.git
%cd DLAM_SealedSurfaces
!pip install -r requirements.txt
%cd src

fatal: destination path 'DLAM_SealedSurfaces' already exists and is not an empty directory.
/content/DLAM_SealedSurfaces
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.1 MB/s eta 0:00:00
  

In [3]:
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW

from settings import *
from dataset import PotsdamDataset, get_data_loaders
from model import custom_resnet18
from train import train_loop

In [10]:
try:
    import os
    from google.colab import drive
    MOUNTPOINT = '/content/drive/'
    drive.mount(MOUNTPOINT)

    IMAGE_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '2_Ortho_RGB') # TODO adjust to your path
    LABEL_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '5_Labels_all') # TODO adjust to your path
except:
    IMAGE_PATH = '../data/2_Ortho_RGB' # TODO adjust to your path
    LABEL_PATH = '../data/5_Labels_all' # TODO adjust to your path

Drive already mounted at /content/dlam_datasets/; to attempt to forcibly remount, call drive.mount("/content/dlam_datasets/", force_remount=True).


In [6]:
# Hyperparameters
patch_size = 250
stride = 250
batch_size = 512
num_epochs = 2
lr = 0.001
SAVE_PATH = MOUNTPOINT + 'MyDrive/DLAM/models/'

# Train resnet18

In [13]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = get_data_loaders(dataset, [0.8, 0.1, 0.1] ,batch_size)

model = custom_resnet18(patch_size=patch_size, device=DEVICE)
loss_fn = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=lr)

train_loop(train_loader=train_loader, val_loader=val_loader, model=model, loss_fn=loss_fn, optimizer=optimizer, num_epochs=num_epochs, device=DEVICE, save_path=SAVE_PATH + 'resnet18.pth')

# Train DeepLabV3 resnet101 baseline

In [ ]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE, no_color_labels=True)
train_loader, val_loader, test_loader = get_data_loaders(dataset, [0.8, 0.1, 0.1] ,batch_size)

model = custom_resnet18(patch_size=patch_size, device=DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=lr)

train_loop(train_loader=train_loader, val_loader=val_loader, model=model, loss_fn=loss_fn, optimizer=optimizer, num_epochs=num_epochs, device=DEVICE, save_path=SAVE_PATH + 'deeplabv3_resnet101.pth')